In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/gdrive/My Drive/Places')

# Let's do this man!

In [0]:
import math, os
import numpy as np
import keras
from keras.layers import Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing import image


PATH = os.getcwd()
# Define data path
DATASET_DIR = PATH + '/my_training_data'

SIZE = (224, 224)
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.3
IMG_SIZE = ( 224, 224, 3)

In [6]:
# How many data and classes ..
num_dataset_samples = sum([len(files) for r, d, files in os.walk(DATASET_DIR)])

np.random.seed(42)
num_train_steps = math.floor((num_dataset_samples*(1-VALIDATION_SPLIT))/BATCH_SIZE)
num_valid_steps = math.floor((num_dataset_samples*VALIDATION_SPLIT)/BATCH_SIZE)

generator = image.ImageDataGenerator(validation_split=VALIDATION_SPLIT)

batches = generator.flow_from_directory(DATASET_DIR, subset='training', target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
val_batches = generator.flow_from_directory(DATASET_DIR, subset='validation', target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)


Found 451 images belonging to 2 classes.
Found 191 images belonging to 2 classes.


In [7]:

#  Let's use mobilenet as it is simple and needs less memory
model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3),  include_top=False, weights='imagenet')


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# classes
classes = list(iter(batches.class_indices))
classes

['bodha', 'nyatapola']

In [9]:

# Our Model starts
x = Sequential()
x.add(model)
x.add(Flatten())
x.add(Dense(16, activation='relu'))
x.add(Dropout(0.5))
x.add(BatchNormalization())
# predictions = Dense(num_classes, activation = 'softmax')(x)
x.add(Dense(len(classes), activation="softmax"))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:

# Compile
x.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [0]:

# logs on csv file... 
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c
x.classes = classes
filename='model_train_new.csv'
csv_log=keras.callbacks.CSVLogger(filename, separator=',', append=False)


In [0]:

# Early stopping if loss doesnt improve
early_stopping = EarlyStopping(patience=10)
checkpointer = ModelCheckpoint('mobile_best.h5', verbose=1, save_best_only=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BATCH_SIZE, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

# Fit and save
x.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=8, callbacks=[early_stopping, checkpointer, tensorboard_callback, csv_log], validation_data=val_batches, validation_steps=num_valid_steps)
x.save('mobile_final.h5')

scoreSeg = x.evaluate_generator(val_batches, num_dataset_samples*VALIDATION_SPLIT )
print("Accuracy = ",scoreSeg[1])

Instructions for updating:
Use tf.cast instead.
Epoch 1/8
28/28 [==============================] - 292s 10s/step - loss: 0.4277 - acc: 0.7924 - val_loss: 0.1411 - val_acc: 0.9634

Epoch 00001: val_loss improved from inf to 0.14106, saving model to mobile_best.h5
Epoch 2/8
28/28 [==============================] - 270s 10s/step - loss: 0.2376 - acc: 0.9263 - val_loss: 0.0841 - val_acc: 0.9948

Epoch 00002: val_loss improved from 0.14106 to 0.08407, saving model to mobile_best.h5
Epoch 3/8
28/28 [==============================] - 273s 10s/step - loss: 0.2221 - acc: 0.9571 - val_loss: 0.0765 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.08407 to 0.07650, saving model to mobile_best.h5
Epoch 4/8
28/28 [==============================] - 270s 10s/step - loss: 0.2037 - acc: 0.9214 - val_loss: 0.0674 - val_acc: 1.0000

Epoch 00004: val_loss improved from 0.07650 to 0.06742, saving model to mobile_best.h5
Epoch 5/8
28/28 [==============================] - 272s 10s/step - loss: 0.1915 

In [6]:
x.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1003536   
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 3,261,618
Trainable params: 3,227,474
Non-trainable params: 34,144
_____________________________________________________________

In [9]:
!pip install tensorflow==1.13.0rc1


     |████████████████████████████████| 92.6MB 2.0MB/s 
     |████████████████████████████████| 3.1MB 44.0MB/s 
     |████████████████████████████████| 368kB 44.5MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-estimator 1.14.0rc0
    Uninstalling tensorflow-estimator-1.14.0rc0:
      Successfully uninstalled tensorflow-estimator-1.14.0rc0
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


In [1]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file( '/gdrive/My Drive/Places/resnet_best.h5') 
tfmodel = converter.convert() 

open ("yes.tflite" , "wb") .write(tfmodel)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 268 variables.
INFO:tensorflow:Converted 268 variables to const ops.


12867328